In [36]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [42]:
pd.set_option('display.max_rows', None)

In [43]:
url = "https://raw.githubusercontent.com/dwightvj/PIC16B-Project/main/dogs.csv"

In [44]:
df = pd.read_csv(url)
# df = df.fillna(0)

In [45]:
df[['id','size', 'lowShedding', 'easyToGroom','goodHealth', 'lowBarking', 'easyToTrain']]

,id,size,lowShedding,easyToGroom,goodHealth,lowBarking,easyToTrain
0,affenpinscher,1,5,3,4,4.0,2
1,afghan-hound,4,2,1,3,4.0,1
2,airedale-terrier,3,4,2,3,2.0,4
3,akita,4,1,1,4,1.0,2
4,alaskan-malamute,4,1,1,4,1.0,4
5,american-english-coonhound,3,3,5,4,1.0,5
6,american-eskimo-dog,2,1,3,5,2.0,5
7,american-foxhound,3,3,5,5,1.0,2
8,american-pit-bull-terrier,3,2,5,4,3.0,4
9,american-water-spaniel,3,3,3,3,3.0,4


In [5]:
df.drop(89, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
# obtain variance for each column
colvar = []
for column in range(2, 14):
    colvar.append(np.var(df.iloc[:, column]))
    print(df.columns[column], np.var(df.iloc[:, column]))

size 1.3438679726558513
kidFriendly 1.08499132741557
dogFriendly 1.08499132741557
lowShedding 1.6023875114784205
easyToGroom 2.1076420773390474
highEnergy 0.9230945821854911
goodHealth 1.152560963167024
lowBarking 1.5219665541498104
intelligence 0.5548668503213957
easyToTrain 1.3194571982450771
toleratesHot 0.8817722681359044
toleratesCold 1.4617130905009692


In [7]:
# only choose top 6 columns of greatest variance, drop the rest
df = df.drop(["name", "dogFriendly", "kidFriendly", "highEnergy",
        "intelligence", "toleratesHot", "toleratesCold"], axis = 1)

In [8]:
df

,id,size,lowShedding,easyToGroom,goodHealth,lowBarking,easyToTrain
0,affenpinscher,1,5,3,4,4.0,2
1,afghan-hound,4,2,1,3,4.0,1
2,airedale-terrier,3,4,2,3,2.0,4
3,akita,4,1,1,4,1.0,2
4,alaskan-malamute,4,1,1,4,1.0,4
...,...,...,...,...,...,...,...
193,whippet,3,4,5,4,5.0,4
194,wirehaired-pointing-griffon,3,5,3,4,2.0,5
195,xoloitzuintli,3,1,5,5,1.0,3
196,yorkipoo,1,5,4,3,1.0,4


In [9]:
# create new vector column for dogs' attributes
df['list'] = df[['size', 'lowShedding', 'easyToGroom',
                'goodHealth', 'lowBarking', 'easyToTrain']].values.tolist()

In [19]:
df[df['id'] == 'Beagle']

,id,size,lowShedding,easyToGroom,goodHealth,lowBarking,easyToTrain,list


In [11]:
from scipy import spatial

#create list of the behavior attributes lists
breeds = df['list'].tolist()
#create KDTree based on these breeds
tree = spatial.KDTree(breeds)


In [12]:
#recommend breed that is the "nearest neighbor" to input
def breedrec(l):
    #calculate nearest neighbor to l
    dog = breeds[tree.query(l)[1]]

    #find the index of this vector of attributes
    index = breeds.index(dog)
    #get breed name based on index
    name = df.iloc[index, 0]
    return name

In [13]:
#recommend the top three breeds that are the "nearest neighbor" to input
def top3rec(l):
    #find the indices of the 3 closest vectors to l
    closest_indices = tree.query(l, k = 3)[1]

    #get the vectors of attributes of these 3 indices
    dogs_behav = [breeds[i] for i in closest_indices]

    #find the indices containing these attribute vectors
    indices = [breeds.index(dog) for dog in dogs_behav]
    #get the breed names based on index
    name = [df.iloc[index,0] for index in indices]
    return name

In [14]:
akita = [4.0, 1.0, 1.0, 4.0, 1.0, 2.0]

breedrec(akita)

'akita'

In [15]:
test = [1, 2, 3, 4, 5, 4]
breedrec(test)

'cardigan-welsh-corgi'

In [16]:
test2 = [1,1,1,1,1,1]
test3 = [5,5,5,5,5,5]
top3rec(test2), top3rec(test3)

(['pekingese', 'petit-basset-griffon-vendeen', 'dachshund'],
 ['belgian-malinois', 'swedish-vallhund', 'saluki'])

In [17]:
top3rec(akita)

['akita', 'finnish-spitz', 'alaskan-malamute']

In [18]:
top3rec(test)

['cardigan-welsh-corgi', 'shetland-sheepdog', 'lancashire-heeler']